# Sand pit

When I worked on AoC 2015, I noticed I can do better than that. I had no idea what to expect, and my approach was horrible. After reading the assignment, I had to search the internet regarding certain features in python. However, I did not want to just sit down and read all the python documentations from the start to the end. So I got some important highlight of the language to focus on. So, I decided to get some hands on training on the toolset that Peter Norvig <a href='https://nbviewer.org/github/norvig/pytudes/blob/main/ipynb/Advent-2021.ipynb'>used</a>. After, I will solve the puzzles, then compare mine with his and see how horrible mine is.

In [2]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

## annotations

<code>annotations</code> delays evaluation of function annotations. Without the future import, NameError is raised below. However, after the import, it works without any issue.

In [1]:
class A:
    def a(self) -> A:
        return

NameError: name 'A' is not defined

## Counter 

Counter can be used to count things.

There are many ways to initiate the Counter.

In [7]:
Counter()

Counter()

In [8]:
Counter('Darkness')

Counter({'D': 1, 'a': 1, 'r': 1, 'k': 1, 'n': 1, 'e': 1, 's': 2})

In [10]:
Counter({'bacon': 4, 'eggs': 30})

Counter({'bacon': 4, 'eggs': 30})

In [15]:
c = Counter(onions=2, cucumber=19, potato=11, sweet_potato=38)
c

Counter({'onions': 2, 'cucumber': 19, 'potato': 11, 'sweet_potato': 38})

Counts can be retrieved by using keys. If it does not exist in the Counter, count is 0.

In [19]:
c['onions'], c['zombie']

(2, 0)

Counts can be changed. It can be 0 or negative. 

In [21]:
c['sweet_potato'] = 0
'sweet_potato' in c

True

In [22]:
c['sweet_potato'] = -100
'sweet_potato' in c

True

Elements in the Counts can be deleted with del. 

In [23]:
del c['sweet_potato']
'sweet_potato' in c

False

Counter has help methods. <code>elements</code> returns an iterator that generates all the elements.

In [26]:
list(c.elements())

['onions',
 'onions',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato']